In [1]:
import requests
from exa_py import Exa
import webbrowser
import torch
from transformers import pipeline
from huggingface_hub import InferenceClient
from joblib import load, dump
from pathlib import Path
from collections import Counter
import json

c:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Keys and inits
with open("_keys.json", 'r') as f:
    keys = json.load(f)  # Loading keys from _keys.json file

EXA_KEY = keys["EXA_KEY"]
HF_KEY = keys["HF_KEY"]

In [16]:
def brainstorm(query, messages=[]):

    if len(messages) < 1: # Creating initial prompts to start the brainstorming session
        system_instruction_init = f"""
        You are a helpful AI research assistant named furhatos-sama.
        Your task is to help the user brainstorm creatively through suggesstions, possibilities and discussions. 

        Instructions for response:
        1. Adjust the session according to each new query of the user.
        2. Your aim is to help the user think creatively from different perspectives.
        3. All messages must be short. There should be no pointwise text in the message.
        4. Do not include any kind of special characters.
        5. Each message will be converted to speech using a text to speech synthesizer so the messages must be formatted accordingly.
        6. This will be a conversational flow, so you can ask questions to the user as well.
        7. Always speak with a first person POV.
        8. Do not insert any tags such as <think> in the output.
        """
        user_prompt_init = f"I need help with brainstorming. Following is my query {query}."

        messages = [
            {
                "role": "system",
                "content": system_instruction_init,
            },
            {"role": "user", "content": user_prompt_init},
        ]
    else:  # Continuing conversation 
        messages.append({"role": "user", "content": query})

    # Querying hugging face LLM using an Inference Client
    client = InferenceClient(api_key=HF_KEY, provider="together")
    completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3", messages=messages, max_tokens=500
    )
    # completion = client.chat.completions.create(
    #     model="HuggingFaceH4/zephyr-7b-beta", messages=messages, max_tokens=500
    # )

    # Processing responce for approapriate TTS by furhat
    llm_response = completion.choices[0].message
    fur_response = llm_response.content.lower().replace("furhatos-sama:", " ").replace("\n", " ")

    print(llm_response)

    # Appending response to messages
    messages.append({"role": "system", "content": fur_response})

    return llm_response, fur_response, messages

In [17]:
query = "I want to work on intersection of ML and Healthcare. Help me think of some good research ideas."
llm_response, fur_response, messages = brainstorm(query = query, messages=[])

ChatCompletionOutputMessage(role='assistant', content='Great topic Let’s explore this. How about focusing on predictive analytics for disease outbreaks using machine learning? You could analyze patterns in health data to predict where outbreaks might occur next. What do you think?', tool_calls=[])


In [5]:
fur_response

"<think> okay, the user wants to brainstorm research ideas at the intersection of ml and healthcare. let me start by recalling some current trends and challenges in both fields. ml in healthcare is a broad area, so narrowing down might be necessary. first, i should consider what aspects of healthcare could benefit most from ml. areas like diagnostics, treatment personalization, predictive analytics, medical imaging, drug discovery, and patient monitoring come to mind.  diagnostics is a big one. ml models can analyze medical images or genetic data to detect diseases early. maybe exploring how to make these models more interpretable for doctors could be a good angle. then there's personalized treatment. using patient data to tailor therapies, perhaps integrating genomics with ml to predict drug responses. that's a hot topic with precision medicine.  another thought: predictive analytics for patient outcomes. hospitals could use ml to predict readmissions or complications. but what data w

In [ ]:
llm_response, fur_response, messages = brainstorm(query = query, messages=messages)
